In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pysar.objects import sensor
from pysar.utils import readfile, plot as pp

work_dir = '~/insarlab/PYSAR_EXAMPLE/AlcedoSenDT128/PYSAR'
work_dir = '~/insarlab/PYSAR_EXAMPLE/SierraNegraAlosAT133/PYSAR'
work_dir = os.path.expanduser(work_dir)
os.chdir(work_dir)
print('Go to directory: '+work_dir)
proj_name = sensor.project_name2sensor_name(work_dir)[1]

# Option 1 - plot with view.py from PySAR
plotCmd = 'view.py avgPhaseVelocity.h5 -m maskSpatialCoh.h5 '
plotCmd += ' -d ./INPUTS/geometryRadar.h5 --dem-nocontour '
plotCmd += ' --fontsize 12 --figsize 4 3 --notitle --dpi 600 --nodisplay --cbar-label "Velocity [cm/yr]" '
plotCmd += ' -o {}_avgPhaseVel.jpg --notick'.format(proj_name)
print(plotCmd)
os.system(plotCmd)

# Option 2 - plot with matplotlib
#data, atr = readfile.read('avgPhaseVelocity.h5')
#mask = readfile.read('maskSpatialCoh.h5')[0]
#data[mask==0] = np.nan
#
#fig, ax = plt.subplots(figsize=[4, 3])
#im = ax.imshow(data, cmap='jet')
#pp.auto_flip_direction(atr, ax=ax, print_msg=False)
#ax.get_xaxis().set_ticks([])
#ax.get_yaxis().set_ticks([])
#
## colorbar
#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", "3%", pad="3%")
#cbar = plt.colorbar(im, cax=cax)
#plt.show()

/Users/yunjunz/development/python/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Go to directory: /Users/yunjunz/insarlab/PYSAR_EXAMPLE/SierraNegraAlosAT133/PYSAR
view.py avgPhaseVelocity.h5 -m maskSpatialCoh.h5  -d ./INPUTS/geometryRadar.h5 --dem-nocontour  --fontsize 12 --figsize 4 3 --notitle --dpi 600 --nodisplay --cbar-label "Velocity [cm/yr]"  -o SierraNegraAlosAT133_avgPhaseVel.jpg --notick


0